Basedo no ViralML e um pouco do estudo do Pardini

Usa um dataset de preços de BHKP e NBSK desde 2017 (pix1])

In [2]:
import pandas as pd
import numpy as np

In [3]:
import logging
log = logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG) 

In [4]:
import importlib, sys, os
from pathlib import Path
sys.path.append( str( Path(os.path.abspath('')).parents[0] ) )

In [6]:
# 
import markov._markov_setup as _markov_setup
from support_library.jupyter_plus import *

_ = importlib.reload(sys.modules['markov._markov_setup'])

# DataCollect

In [8]:
df_master = pd.read_csv("pix1.csv")

df = df_master[['DATE','PIX_CHINA_BHKP_NET_USDT', 'PIX_CHINA_NBSK_NET_USDT','year','week']]
df = df.rename(columns = {'PIX_CHINA_BHKP_NET_USDT':'bhkp_price', 
                          'PIX_CHINA_NBSK_NET_USDT':'nbkp_price', 
                          "DATE":'date'})

df['date'] = pd.to_datetime(df['date'], infer_datetime_format = True)

df['bhkp_delta'] = df['bhkp_price'].diff()
df['bhkp_delta_outcome'] = df['bhkp_price'].diff(-1)
df['bhkp_variation'] = df['bhkp_price'].pct_change()

df['nbkp_delta'] = df['nbkp_price'].diff()
df['nbkp_variation'] = df['nbkp_price'].pct_change()
df = df.fillna(0)

df

df = df[['date','year','week', 'bhkp_price', 'bhkp_delta', 'bhkp_variation', 'nbkp_price', 'nbkp_delta', 'nbkp_variation']] 

In [9]:
print(df.shape)
print(df.dtypes)
df.head()

(253, 9)
date              datetime64[ns, UTC]
year                            int64
week                            int64
bhkp_price                    float64
bhkp_delta                    float64
bhkp_variation                float64
nbkp_price                    float64
nbkp_delta                    float64
nbkp_variation                float64
dtype: object


,date,year,week,bhkp_price,bhkp_delta,bhkp_variation,nbkp_price,nbkp_delta,nbkp_variation
0,2017-04-04 00:00:00+00:00,2017,14,592.08,0.00,0.000000,647.76,0.00,0.000000
1,2017-04-11 00:00:00+00:00,2017,15,598.47,6.39,0.010792,644.48,-3.28,-0.005064
2,2017-04-18 00:00:00+00:00,2017,16,600.08,1.61,0.002690,642.52,-1.96,-0.003041
3,2017-04-25 00:00:00+00:00,2017,17,610.29,10.21,0.017014,646.20,3.68,0.005727
4,2017-05-02 00:00:00+00:00,2017,18,613.88,3.59,0.005882,647.22,1.02,0.001578


In [10]:
df[['bhkp_price', 'bhkp_delta', 'bhkp_variation', 'nbkp_price', 'nbkp_delta', 'nbkp_variation']].describe()

,bhkp_price,bhkp_delta,bhkp_variation,nbkp_price,nbkp_delta,nbkp_variation
count,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000
mean,616.829051,0.042451,0.000231,730.434704,0.731502,0.001152
std,120.906163,11.403620,0.018161,135.485971,13.677085,0.017938
min,441.260000,-58.140000,-0.078251,554.410000,-67.660000,-0.085567
25%,471.030000,-2.000000,-0.003672,584.750000,-2.860000,-0.003910
50%,631.140000,0.040000,0.000084,708.340000,-0.340000,-0.000470
75%,742.990000,2.200000,0.003820,855.540000,2.830000,0.004170
max,780.090000,97.260000,0.157578,985.230000,90.890000,0.107113


# Markov

## Random Sets
- **Outcome_Next_Day_Direction**: Resultado Direção do dia seguinte

In [11]:
# from markov._markov_setup import *
import markov._markov_setup as _markov_setup
_ = importlib.reload(sys.modules['markov._markov_setup'])

In [12]:
# %time my_set = _markov_setup.get_random_sets(df, 2, \
#                                              columns=['bhkp_price', 'bhkp_delta', 'bhkp_variation', 'nbkp_price', 'nbkp_delta', 'nbkp_variation'], \
#                                              outcomes=['bhkp_price', 'bhkp_delta', 'bhkp_variation'])


%time my_set = _markov_setup.get_random_sets(df, 10000, \
                                             columns=['bhkp_price','nbkp_price'], \
                                             outcomes=['bhkp_price', 'nbkp_price'], \
                                             bypass=[])

get_random_sets:   0%|▏                                                 | 26/10000 [00:00<00:39, 254.90it/s]

row_set: 0      | row_quant: 12 | rows:    168-179    | market_subset:  (12, 9)   | Close_Date: 09/04/2020


get_random_sets:  20%|██████████▏                                       | 2039/10000 [00:07<00:30, 258.47it/s]

row_set: 2000   | row_quant: 27 | rows:    124-150    | market_subset:  (27, 9)   | Close_Date: 02/18/2020


get_random_sets:  40%|████████████████████▏                             | 4046/10000 [00:14<00:18, 313.82it/s]

...

row_set: 6000   | row_quant: 12 | rows:    114-125    | market_subset:  (12, 9)   | Close_Date: 08/27/2019


get_random_sets:  81%|████████████████████████████████████████▎         | 8051/10000 [00:28<00:06, 300.45it/s]

row_set: 8000   | row_quant: 30 | rows:    186-215    | market_subset:  (30, 9)   | Close_Date: 05/14/2021


get_random_sets: 100%|██████████████████████████████████████████████████| 10000/10000 [00:35<00:00, 284.42it/s]

CPU times: total: 38.6 s
Wall time: 35.2 s


In [13]:
%time df1 = _markov_setup.convet_collection_to_dataframe(my_set)
df1

DEBUG:markov._markov_setup:Before dropping NaNs: (200445, 9)
DEBUG:markov._markov_setup:After dropping NaNs: (180445, 9)


CPU times: total: 1.25 s
Wall time: 1.25 s


,Sequence_ID,Close_Date,ref_date,bhkp_price,bhkp_price_variation,bhkp_price_outcome,nbkp_price,nbkp_price_variation,nbkp_price_outcome
169,0,2020-09-04 00:00:00+00:00,2020-06-30 00:00:00+00:00,460.54,-0.007585,-14.65,565.85,-0.000636,-10.47
170,0,2020-09-04 00:00:00+00:00,2020-07-07 00:00:00+00:00,445.89,-0.031810,-2.00,555.38,-0.018503,-0.97
171,0,2020-09-04 00:00:00+00:00,2020-07-14 00:00:00+00:00,443.89,-0.004485,1.52,554.41,-0.001747,3.77
172,0,2020-09-04 00:00:00+00:00,2020-07-21 00:00:00+00:00,445.41,0.003424,-2.47,558.18,0.006800,4.18
173,0,2020-09-04 00:00:00+00:00,2020-07-28 00:00:00+00:00,442.94,-0.005545,-0.63,562.36,0.007489,3.05
...,...,...,...,...,...,...,...,...,...
107,9999,2019-05-28 00:00:00+00:00,2019-04-23 00:00:00+00:00,686.28,-0.003224,-7.68,709.42,-0.004295,-5.14
108,9999,2019-05-28 00:00:00+00:00,2019-04-30 00:00:00+00:00,678.60,-0.011191,-6.44,704.28,-0.007245,-7.18
109,9999,2019-05-28 00:00:00+00:00,2019-05-07 00:00:00+00:00,672.16,-0.009490,-4.33,697.10,-0.010195,-5.07
110,9999,2019-05-28 00:00:00+00:00,2019-05-14 00:00:00+00:00,667.83,-0.006442,-4.13,692.03,-0.007273,-8.73


In [14]:
_markov_setup.export_dataframe(df1, 'step_001_100k')

In [15]:
df2 = _markov_setup.import_dataframe('step_001_100k')
df2

,Sequence_ID,Close_Date,ref_date,bhkp_price,bhkp_price_variation,bhkp_price_outcome,nbkp_price,nbkp_price_variation,nbkp_price_outcome
169,0,2020-09-04 00:00:00+00:00,2020-06-30 00:00:00+00:00,460.54,-0.007585,-14.65,565.85,-0.000636,-10.47
170,0,2020-09-04 00:00:00+00:00,2020-07-07 00:00:00+00:00,445.89,-0.031810,-2.00,555.38,-0.018503,-0.97
171,0,2020-09-04 00:00:00+00:00,2020-07-14 00:00:00+00:00,443.89,-0.004485,1.52,554.41,-0.001747,3.77
172,0,2020-09-04 00:00:00+00:00,2020-07-21 00:00:00+00:00,445.41,0.003424,-2.47,558.18,0.006800,4.18
173,0,2020-09-04 00:00:00+00:00,2020-07-28 00:00:00+00:00,442.94,-0.005545,-0.63,562.36,0.007489,3.05
...,...,...,...,...,...,...,...,...,...
107,9999,2019-05-28 00:00:00+00:00,2019-04-23 00:00:00+00:00,686.28,-0.003224,-7.68,709.42,-0.004295,-5.14
108,9999,2019-05-28 00:00:00+00:00,2019-04-30 00:00:00+00:00,678.60,-0.011191,-6.44,704.28,-0.007245,-7.18
109,9999,2019-05-28 00:00:00+00:00,2019-05-07 00:00:00+00:00,672.16,-0.009490,-4.33,697.10,-0.010195,-5.07
110,9999,2019-05-28 00:00:00+00:00,2019-05-14 00:00:00+00:00,667.83,-0.006442,-4.13,692.03,-0.007273,-8.73


In [14]:
red('me pegunto se é efetivo criar o tal do random sets... quando aplico o groupby quase que volta ao tamanho original... mas vamos ver no que vai dar')

df2.groupby(['ref_date','bhkp_price', 'bhkp_price_variation','bhkp_price_outcome','nbkp_price','nbkp_price_variation','nbkp_price_outcome']).count()
# df2.groupby(['ref_date']).count()

<span style='color:red'>me pegunto se é efetivo criar o tal do random sets... quando aplico o groupby quase que volta ao tamanho original... mas vamos ver no que vai dar</span>

,,,,,,,Sequence_ID,Close_Date
ref_date,bhkp_price,bhkp_price_variation,bhkp_price_outcome,nbkp_price,nbkp_price_variation,nbkp_price_outcome,,
2017-04-11,598.47,0.010792,1.61,644.48,-0.005064,-1.96,38,38
2017-04-18,600.08,0.002690,10.21,642.52,-0.003041,3.68,89,89
2017-04-25,610.29,0.017014,3.59,646.20,0.005727,1.02,127,127
2017-05-02,613.88,0.005882,2.98,647.22,0.001578,-0.03,174,174
2017-05-09,616.86,0.004854,1.58,647.19,-0.000046,-0.21,221,221
...,...,...,...,...,...,...,...,...
2021-12-24,570.73,0.006525,5.69,757.82,-0.012252,-2.77,221,221
2021-12-31,576.42,0.009970,11.76,755.05,-0.003655,7.07,173,173
2022-01-07,588.18,0.020402,5.11,762.12,0.009364,17.14,129,129


## Divisao do dataset em Quantis



In [15]:
# from markov._markov_setup import *
import markov._markov_setup as _markov_setup
_ = importlib.reload(sys.modules['markov._markov_setup'])

In [16]:
qtd_bins = 5

blue(f"A quantidade de Quantiles (bins) é {qtd_bins}")
df3 = _markov_setup.quantiles(df2, qtd_bins, \
                              columns=['bhkp_price_variation','nbkp_price_variation'], \
                              event_pattern=True)
df3

<span style='color:blue'>A quantidade de Quantiles (bins) é 5</span>

DEBUG:markov._markov_setup:cut_intervals of bhkp_price_variation: [-0.07825139 -0.00463347 -0.00126149  0.00082365  0.004392    0.15757753]
DEBUG:markov._markov_setup:cut_intervals of nbkp_price_variation: [-0.0855665  -0.00480872 -0.00177964  0.00064427  0.00558913  0.10711339]
DEBUG:markov._markov_setup:Unique event_patterns: ['EE' 'DD' 'DB' 'DC' 'EB' 'CC' 'CB' 'CA' 'BB' 'BD' 'BE' 'AA' 'AD' 'AB'
 'AE' 'AC' 'BC' 'DE' 'ED' 'EA' 'BA' 'EC' 'CD' 'CE' 'DA']


,Sequence_ID,Close_Date,ref_date,bhkp_price,bhkp_price_variation,bhkp_price_variation_label,bhkp_price_outcome,nbkp_price,nbkp_price_variation,nbkp_price_variation_label,nbkp_price_outcome,event_pattern
194,0,2021-02-12,2020-12-18,485.29,0.038142,E,13.79,659.09,0.050125,E,11.47,EE
195,0,2021-02-12,2020-12-29,499.08,0.028416,E,1.38,670.56,0.017403,E,2.32,EE
196,0,2021-02-12,2021-01-05,500.46,0.002765,D,18.26,672.88,0.003460,D,13.12,DD
197,0,2021-02-12,2021-01-08,518.72,0.036486,E,15.22,686.00,0.019498,E,56.22,EE
198,0,2021-02-12,2021-01-15,533.94,0.029341,E,2.97,742.22,0.081953,E,36.35,EE
...,...,...,...,...,...,...,...,...,...,...,...,...
138,9999,2019-12-31,2019-11-26,458.29,0.000131,C,-0.96,562.72,-0.016774,A,-1.12,CA
139,9999,2019-12-31,2019-12-03,457.33,-0.002095,B,-0.12,561.60,-0.001990,B,1.90,BB
140,9999,2019-12-31,2019-12-10,457.21,-0.000262,C,0.39,563.50,0.003383,D,-0.25,CD
141,9999,2019-12-31,2019-12-17,457.60,0.000853,D,-0.91,563.25,-0.000444,C,-3.49,DC


In [17]:
df3[['bhkp_price','bhkp_price_variation','bhkp_price_outcome','bhkp_price_variation_label']].describe()

,bhkp_price,bhkp_price_variation,bhkp_price_outcome
count,180062.000000,180062.000000,180062.000000
mean,620.349152,-0.000057,-0.156644
std,125.681640,0.018609,11.732972
min,441.260000,-0.078251,-58.140000
25%,467.460000,-0.003771,-2.170000
50%,642.500000,-0.000026,-0.020000
75%,757.570000,0.003086,1.720000
max,780.090000,0.157578,97.260000
